In [8]:
import cv2
import time
import os
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from playsound import playsound

# -----------------------------
# Landmark indices
# -----------------------------
NOSE = 0
LEFT_EAR = 7
RIGHT_EAR = 8
LEFT_SHOULDER = 11
RIGHT_SHOULDER = 12

# -----------------------------
# Helper functions
# -----------------------------
def angle_between_vectors(v1, v2):
    """
    Returns angle (degrees) between two 3D vectors
    """
    v1, v2 = np.array(v1), np.array(v2)
    cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2) + 1e-6)
    return np.degrees(np.arccos(np.clip(cos_angle, -1, 1)))

def lm3d(landmark):
    """
    Convert a landmark to 3D NumPy array
    """
    return np.array([landmark.x, landmark.y, landmark.z])

def draw_skeleton(frame, landmarks, color=(0, 255, 0)):
    """
    Draw upper-body skeleton lines
    """
    h, w, _ = frame.shape
    def to_pixel(l):
        return int(l.x * w), int(l.y * h)
    
    left_sh = to_pixel(landmarks[LEFT_SHOULDER])
    right_sh = to_pixel(landmarks[RIGHT_SHOULDER])
    nose = to_pixel(landmarks[NOSE])
    left_ear = to_pixel(landmarks[LEFT_EAR])
    right_ear = to_pixel(landmarks[RIGHT_EAR])

    # Shoulder line
    cv2.line(frame, left_sh, right_sh, color, 2)
    # Nose to mid-shoulder
    mid_sh = ((left_sh[0]+right_sh[0])//2, (left_sh[1]+right_sh[1])//2)
    cv2.line(frame, nose, mid_sh, color, 2)
    # Optional: head connections
    cv2.line(frame, nose, left_ear, color, 1)
    cv2.line(frame, nose, right_ear, color, 1)
    # Draw landmarks
    for p in [left_sh, right_sh, nose, left_ear, right_ear]:
        cv2.circle(frame, p, 5, color, -1)

# -----------------------------
# Config
# -----------------------------
sound_file = "alert.wav"
alert_cooldown = 5

is_calibrated = False
calibration_frames = 0
head_tilts = []
back_tilts = []
last_alert_time = 0

# -----------------------------
# MediaPipe PoseLandmarker setup
# -----------------------------
BaseOptions = python.BaseOptions
PoseLandmarker = vision.PoseLandmarker
PoseLandmarkerOptions = vision.PoseLandmarkerOptions
RunningMode = vision.RunningMode

model_path = "pose_landmarker_lite.task"
assert os.path.exists(model_path), "pose_landmarker_lite.task not found!"

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=RunningMode.VIDEO,
    num_poses=1,
    min_pose_detection_confidence=0.5,
    min_pose_presence_confidence=0.5,
    min_tracking_confidence=0.5
)

pose = PoseLandmarker.create_from_options(options)

# -----------------------------
# Start webcam
# -----------------------------
cap = cv2.VideoCapture(0)

# -----------------------------
# Main loop
# -----------------------------
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    timestamp_ms = int(time.time()*1000)

    results = pose.detect_for_video(mp_image, timestamp_ms)

    if results.pose_landmarks:
        l = results.pose_landmarks[0]

        left_sh = lm3d(l[LEFT_SHOULDER])
        right_sh = lm3d(l[RIGHT_SHOULDER])
        nose = lm3d(l[NOSE])

        # Mid-shoulder
        mid_shoulder = (left_sh + right_sh)/2

        # -------------------------
        # Posture metrics
        # -------------------------
        # Forward head / slouch: angle nose -> mid-shoulder vs vertical
        head_vec = mid_shoulder - nose
        head_tilt = angle_between_vectors(head_vec, [0, -1, 0])

        # Upper back tilt: shoulder line vs horizontal
        shoulder_vec = right_sh - left_sh
        back_tilt = angle_between_vectors(shoulder_vec, [1, 0, 0])

        # -------------------------
        # Calibration
        # -------------------------
        if not is_calibrated and calibration_frames < 30:
            head_tilts.append(head_tilt)
            back_tilts.append(back_tilt)
            calibration_frames += 1
            cv2.putText(frame, f"Calibrating {calibration_frames}/30", (10,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        elif not is_calibrated:
            head_thresh = np.mean(head_tilts) + 20
            back_thresh = np.mean(back_tilts) + 20
            is_calibrated = True
            print(f"Calibration done. head_thresh={head_thresh:.1f}, back_thresh={back_thresh:.1f}")

        # -------------------------
        # Feedback
        # -------------------------
        if is_calibrated:
            poor_posture = head_tilt > head_thresh or back_tilt < back_thresh
            now = time.time()
            if poor_posture:
                status = "Poor Posture"
                color = (0,0,255)
                if now - last_alert_time > alert_cooldown:
                    print("Alert: Poor posture!")
                    if os.path.exists(sound_file):
                        playsound(sound_file)
                    last_alert_time = now
            else:
                status = "Good Posture"
                color = (0,255,0)

            # Draw skeleton and angles
            draw_skeleton(frame, l, color=color)
            cv2.putText(frame, status, (10,30), cv2.FONT_HERSHEY_SIMPLEX,1,color,2)
            cv2.putText(frame, f"Head tilt: {head_tilt:.1f}", (10,60),
                        cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),1)
            cv2.putText(frame, f"Back tilt: {back_tilt:.1f}", (10,90),
                        cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),1)

    # Show frame
    cv2.imshow("Table Posture Detector", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1768023693.061025  246737 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M3 Max
W0000 00:00:1768023693.130631  246739 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768023693.137292  246739 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Calibration done. head_thresh=137.9, back_thresh=178.6
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!
Alert: Poor posture!


: 